In [31]:
import os
from google.colab import userdata
import requests
import hashlib
import time
import pandas as pd
from tqdm import tqdm
from pprint import pprint as pp

public_key=userdata.get('public_key')
private_key=userdata.get('private_key')


ts = str(time.time())
to_hash = ts + str(private_key) + str(public_key)
hash_md5 = hashlib.md5(to_hash.encode('utf-8')).hexdigest()

characters = "https://gateway.marvel.com/v1/public/characters"
comics = "https://gateway.marvel.com/v1/public/comics"
creators = "http://gateway.marvel.com/v1/public/creators"

params = {
    "ts": ts,
    "apikey": public_key,
    "hash": hash_md5,
    "limit": 100
}




# FUNCTIONS

In [7]:
def get_data(url, public_key, private_key, limit_num, offset_num):
  '''
  url: link de requisição da API
  public_key: chave pública gerada pela API da marvel
  private_key: chave privada gerada pela API da marvel
  limit_num: quantidade de linhas que serão pegas ao realizar requisição
  offset_num: disposição em relação a posição inicial

  return: df_normalizado: Dataframe somente com dados necessários da API
  '''


  ts = str(time.time())
  to_hash = ts + str(private_key) + str(public_key)
  hash_md5 = hashlib.md5(to_hash.encode('utf-8')).hexdigest()

  params = {
    "ts": ts,
    "apikey": public_key,
    "hash": hash_md5,
    "limit": limit_num,
    "offset": offset_num
  }

  response = requests.get(url, params=params)
  data = response.json()
  df = pd.DataFrame(data)
  df_normalizado = pd.json_normalize(df["data"]["results"])
  return df_normalizado

In [8]:
def get_data_and_total(url, public_key, private_key, limit_num, offset_num):
  '''
  url: link de requisição da API
  public_key: chave pública gerada pela API da marvel
  private_key: chave privada gerada pela API da marvel
  limit_num: quantidade de linhas que serão pegas ao realizar requisição
  offset_num: disposição em relação a posição inicial

  return: df_normalizado: Dataframe somente com dados necessários da API
  df["data"]["total"]: quantidade total de linhas na API
  '''

  ts = str(time.time())
  to_hash = ts + str(private_key) + str(public_key)
  hash_md5 = hashlib.md5(to_hash.encode('utf-8')).hexdigest()

  params = {
    "ts": ts,
    "apikey": public_key,
    "hash": hash_md5,
    "limit": limit_num,
    "offset": offset_num
  }

  response = requests.get(url, params=params)
  data = response.json()
  df = pd.DataFrame(data)
  df_normalizado = pd.json_normalize(df["data"]["results"])
  return df_normalizado, df["data"]["total"]

In [23]:
def get_all_data(url):
  '''
  url: link de requisição da API
  return: retorna todos os dados da API inserida
  '''

  partial_data, total_amount = get_data_and_total(url, public_key, private_key, 100, 0)
  #Cria df de todos os dados com colunas
  all_data = pd.DataFrame(columns=partial_data.columns)

  step = 100
  total_steps = (total_amount + step - 1) // step

  offset_get_all = 0

  for offset in tqdm(range(0, total_steps * step, step), desc="Baixando dados da Marvel"):

    batch_data = get_data(url, public_key, private_key, step, offset)
    all_data = pd.concat([all_data, batch_data], ignore_index=True)

  return all_data


In [ ]:
# TO DO: Adicionar esse total no GET ALL
#data["data"]["total"]

64269

# CHARACTERS

In [14]:
limit_num = 100
offset_num = 0

personagens = get_data(characters, public_key, private_key, limit_num, offset_num)

In [15]:
personagens.columns

Index(['id', 'name', 'description', 'modified', 'resourceURI', 'urls',
       'thumbnail.path', 'thumbnail.extension', 'comics.available',
       'comics.collectionURI', 'comics.items', 'comics.returned',
       'series.available', 'series.collectionURI', 'series.items',
       'series.returned', 'stories.available', 'stories.collectionURI',
       'stories.items', 'stories.returned', 'events.available',
       'events.collectionURI', 'events.items', 'events.returned'],
      dtype='object')

In [ ]:
personagens.head()

,id,name,description,modified,resourceURI,urls,thumbnail.path,thumbnail.extension,comics.available,comics.collectionURI,...,series.items,series.returned,stories.available,stories.collectionURI,stories.items,stories.returned,events.available,events.collectionURI,events.items,events.returned
0,1011334,3-D Man,,2014-04-29T14:18:17+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/c/e0/53...,jpg,12,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,3,21,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,1,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,1
1,1017100,A-Bomb (HAS),Rick Jones has been Hulk's best bud since day ...,2013-09-18T15:54:04+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/3/20/52...,jpg,4,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,7,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,7,0,https://gateway.marvel.com/v1/public/character...,[],0
2,1009144,A.I.M.,AIM is a terrorist organization bent on destro...,2013-10-17T14:41:30+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/6/20/52...,jpg,53,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,61,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,0,https://gateway.marvel.com/v1/public/character...,[],0
3,1010699,Aaron Stack,,1970-01-01T00:00:00+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/b/40/im...,jpg,14,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,3,28,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,0,https://gateway.marvel.com/v1/public/character...,[],0
4,1009146,Abomination (Emil Blonsky),"Formerly known as Emil Blonsky, a spy of Sovie...",2014-06-27T19:39:07+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/9/50/4c...,jpg,55,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,68,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,1,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,1


# COMICS

In [ ]:
limit_num = 100
offset_num = 0

quadrinhos = get_data(comics, public_key, private_key, limit_num, offset_num)

In [ ]:
quadrinhos.columns

Index(['id', 'digitalId', 'title', 'issueNumber', 'variantDescription',
       'description', 'modified', 'isbn', 'upc', 'diamondCode', 'ean', 'issn',
       'format', 'pageCount', 'textObjects', 'resourceURI', 'urls', 'variants',
       'collections', 'collectedIssues', 'dates', 'prices', 'images',
       'series.resourceURI', 'series.name', 'thumbnail.path',
       'thumbnail.extension', 'creators.available', 'creators.collectionURI',
       'creators.items', 'creators.returned', 'characters.available',
       'characters.collectionURI', 'characters.items', 'characters.returned',
       'stories.available', 'stories.collectionURI', 'stories.items',
       'stories.returned', 'events.available', 'events.collectionURI',
       'events.items', 'events.returned'],
      dtype='object')

In [ ]:
quadrinhos.head()

,id,digitalId,title,issueNumber,variantDescription,description,modified,isbn,upc,diamondCode,...,characters.items,characters.returned,stories.available,stories.collectionURI,stories.items,stories.returned,events.available,events.collectionURI,events.items,events.returned
0,82967,0,Marvel Previews (2017),0,,,2019-11-07T08:46:15+0000,,75960608839302811,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/82...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/82...,[],0
1,82965,0,Marvel Previews (2017),0,,,2019-08-21T17:11:27+0000,,75960608839302611,JUL190068,...,[],0,2,https://gateway.marvel.com/v1/public/comics/82...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/82...,[],0
2,82970,52952,Marvel Previews (2017),0,,,2020-02-07T09:35:32+0000,,75960608839303111,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/82...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/82...,[],0
3,61137,0,The Punisher (2016) #5 (Cosplay Variant),5,Cosplay Variant,None,2016-09-01T12:40:16+0000,,75960608413500521,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/61...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/61...,[],0
4,37497,0,MARVELS (1994) #1,1,,None,2014-05-08T12:18:55+0000,,,,...,[],0,0,https://gateway.marvel.com/v1/public/comics/37...,[],0,0,https://gateway.marvel.com/v1/public/comics/37...,[],0


# GET ALL

## GET ALL CHARACTERS

In [24]:
todos_personagens = get_all_data(characters)
todos_personagens

Baixando dados da Marvel: 100%|██████████| 16/16 [00:29<00:00,  1.83s/it]


,id,name,description,modified,resourceURI,urls,thumbnail.path,thumbnail.extension,comics.available,comics.collectionURI,...,series.items,series.returned,stories.available,stories.collectionURI,stories.items,stories.returned,events.available,events.collectionURI,events.items,events.returned
0,1011334,3-D Man,,2014-04-29T14:18:17+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/c/e0/53...,jpg,12,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,3,21,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,1,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,1
1,1017100,A-Bomb (HAS),Rick Jones has been Hulk's best bud since day ...,2013-09-18T15:54:04+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/3/20/52...,jpg,4,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,7,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,7,0,https://gateway.marvel.com/v1/public/character...,[],0
2,1009144,A.I.M.,AIM is a terrorist organization bent on destro...,2013-10-17T14:41:30+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/6/20/52...,jpg,53,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,61,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,0,https://gateway.marvel.com/v1/public/character...,[],0
3,1010699,Aaron Stack,,1970-01-01T00:00:00+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/b/40/im...,jpg,14,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,3,28,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,0,https://gateway.marvel.com/v1/public/character...,[],0
4,1009146,Abomination (Emil Blonsky),"Formerly known as Emil Blonsky, a spy of Sovie...",2014-06-27T19:39:07+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/9/50/4c...,jpg,55,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,68,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,1,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1559,1011515,Zeus,Zeus ruled a peaceful Olympus for centuries wh...,2021-08-18T22:06:56+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/f/60/4c...,png,36,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,13,43,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,0,https://gateway.marvel.com/v1/public/character...,[],0
1560,1011127,Zodiak,"Twelve demons merged with Norman Harrison, who...",1970-01-01T00:00:00+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/b/40/im...,jpg,1,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,1,1,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,1,0,htt

In [25]:
todos_personagens.to_csv("TodosPersonagens.csv")

## GET ALL COMICS

In [26]:
todos_comics = get_all_data(comics)


Baixando dados da Marvel: 100%|██████████| 643/643 [42:20<00:00,  3.95s/it]


In [27]:
todos_comics.head()

,id,digitalId,title,issueNumber,variantDescription,description,modified,isbn,upc,diamondCode,...,characters.items,characters.returned,stories.available,stories.collectionURI,stories.items,stories.returned,events.available,events.collectionURI,events.items,events.returned
0,82967,0,Marvel Previews (2017),0,,,2019-11-07T08:46:15+0000,,75960608839302811,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/82...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/82...,[],0
1,82965,0,Marvel Previews (2017),0,,,2019-08-21T17:11:27+0000,,75960608839302611,JUL190068,...,[],0,2,https://gateway.marvel.com/v1/public/comics/82...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/82...,[],0
2,82970,52952,Marvel Previews (2017),0,,,2020-02-07T09:35:32+0000,,75960608839303111,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/82...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/82...,[],0
3,61137,0,The Punisher (2016) #5 (Cosplay Variant),5,Cosplay Variant,None,2016-09-01T12:40:16+0000,,75960608413500521,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/61...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/61...,[],0
4,37497,0,MARVELS (1994) #1,1,,None,2014-05-08T12:18:55+0000,,,,...,[],0,0,https://gateway.marvel.com/v1/public/comics/37...,[],0,0,https://gateway.marvel.com/v1/public/comics/37...,[],0


In [29]:
todos_comics.to_csv("TodosComics.csv")

In [30]:
todos_comics

,id,digitalId,title,issueNumber,variantDescription,description,modified,isbn,upc,diamondCode,...,characters.items,characters.returned,stories.available,stories.collectionURI,stories.items,stories.returned,events.available,events.collectionURI,events.items,events.returned
0,82967,0,Marvel Previews (2017),0,,,2019-11-07T08:46:15+0000,,75960608839302811,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/82...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/82...,[],0
1,82965,0,Marvel Previews (2017),0,,,2019-08-21T17:11:27+0000,,75960608839302611,JUL190068,...,[],0,2,https://gateway.marvel.com/v1/public/comics/82...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/82...,[],0
2,82970,52952,Marvel Previews (2017),0,,,2020-02-07T09:35:32+0000,,75960608839303111,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/82...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/82...,[],0
3,61137,0,The Punisher (2016) #5 (Cosplay Variant),5,Cosplay Variant,None,2016-09-01T12:40:16+0000,,75960608413500521,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/61...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/61...,[],0
4,37497,0,MARVELS (1994) #1,1,,None,2014-05-08T12:18:55+0000,,,,...,[],0,0,https://gateway.marvel.com/v1/public/comics/37...,[],0,0,https://gateway.marvel.com/v1/public/comics/37...,[],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64263,21671,0,Psi-Force (1986) #28,28,,None,-0001-11-30T00:00:00+0000,,,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/21...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/21...,[],0
64264,21672,0,Psi-Force (1986) #29,29,,None,-0001-11-30T00:00:00+0000,,,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/21...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/21...,[],0
64265,21674,0,Psi-Force (1986) #30,30,,None,-0001-11-30T00:00:00+0000,,,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/21...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/21...,[],0
64266,21675,0,Psi-Force (1986) #31,31,,None,-0001-11-30T00:00:00+0000,,,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/21...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/21...,[],0


## GET ALL CREATORS

In [32]:
todos_creators = get_all_data(creators)

ConnectionError: HTTPConnectionPool(host='gateway.marvel.com', port=80): Max retries exceeded with url: /v1/public/creators?ts=1747311214.2105792&apikey=8ed993fa80600a97b969f62b58729d0c&hash=98d8c6eb7575935b481d10c845c11228&limit=100&offset=0 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7cb9f884f210>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [33]:
todos_creators.head()

NameError: name 'todos_creators' is not defined

In [ ]:
todos_creators.to_csv("TodosCreators.csv")